In [9]:
!jt -t grade3 -fs 90 -tfs 15 -nfs 115 -cellw 80% -T
import ROOT as R
import array
import numpy as np
import pandas as pd
import pickle
import datetime
%jsroot on
c = R.TCanvas()
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm


In [10]:
import log_loader_time

In [11]:
chain=R.TChain("tree")
chain.Add("/media/alb/Removibile/dati/351/Sub_RUN_ana_*")


990

In [12]:
chunck_size=chain.GetEntries()


In [13]:
red=log_loader_time.reader("/media/alb/Removibile/dati_raw/", "/media/alb/Removibile/dati/")


input_line_73:1:8: error: redefinition of 'TreeStruct'
struct TreeStruct {                int runNo;                int subRunNo;                int triggers...
       ^
input_line_68:1:8: note: previous definition is here
struct TreeStruct {                int runNo;                int subRunNo;                int triggers...
       ^


In [14]:
dict_time=red.elab_on_run_dict(351)

In [17]:
def get_number_of_trig_per_sub(num):
    
    tree=R.TChain("tree")
    tree.Add("/media/alb/Removibile/dati/{}/Sub_RUN_ana_*".format(num))

    num_trig_strart = {}
    num_trig_end = {}
    num_trig_per_sub = {}
    hit_noise_per_sub={}
    hit_charge_per_sub={}
    
    for entryNum in tqdm(range (0,chunck_size)):
        #parte trigger
        tree.GetEntry(entryNum)
        subRunNo=getattr(tree ,"subRunNo")
        count=getattr(tree ,"count")
        if subRunNo in num_trig_strart.keys():
            if count < num_trig_strart[subRunNo]:
                num_trig_strart[subRunNo]=count
            if count > num_trig_end[subRunNo]:
                num_trig_end[subRunNo]=count
        else:
            num_trig_strart[subRunNo]=count
            num_trig_end[subRunNo]=count

        #parte hit
        subRunNo=getattr(tree ,"subRunNo")
        charge_SH=getattr(tree ,"charge_SH")
        delta_coarse=getattr(tree ,"delta_coarse")
        l1ts_min_tcoarse=getattr(tree ,"l1ts_min_tcoarse")
        if delta_coarse==25 or delta_coarse==26:
            if (l1ts_min_tcoarse<1370 or l1ts_min_tcoarse>1430):
                if subRunNo in hit_noise_per_sub.keys():
                    hit_noise_per_sub[subRunNo]+=1
                else:
                    hit_noise_per_sub[subRunNo]=0
            elif (l1ts_min_tcoarse>1380 and l1ts_min_tcoarse<1420) and charge_SH>30:
                if subRunNo in hit_charge_per_sub.keys():
                    hit_charge_per_sub[subRunNo]+=1
                else:
                    hit_charge_per_sub[subRunNo]=0

    tot=0
    for key in num_trig_strart.keys():
        tot += num_trig_end[key]-num_trig_strart[key]
        num_trig_per_sub[key]=num_trig_end[key]-num_trig_strart[key]

    return tot,num_trig_per_sub,hit_noise_per_sub,hit_charge_per_sub

In [18]:
 tot,num_trig_per_sub,hit_noise_per_sub,hit_charge_per_sub=get_number_of_trig_per_sub(351)

100%|██████████| 25448176/25448176 [10:48<00:00, 39240.83it/s]


In [20]:
data_4_pd={
    "sub_num":[],
    "charge_hits":[],
    "noise_hits":[],
    "time":[],
    "triggers":[],
    "error_charge":[],
    "error_noise":[]

}
for key in num_trig_per_sub:
    if  key in num_trig_per_sub.keys() and key in hit_charge_per_sub.keys() and key in hit_noise_per_sub.keys() and num_trig_per_sub[key]>50:
        data_4_pd["sub_num"].append(key)
        data_4_pd["charge_hits"].append(hit_charge_per_sub[key]/num_trig_per_sub[key])
        data_4_pd["noise_hits"].append(hit_noise_per_sub[key]/num_trig_per_sub[key])
        data_4_pd["time"].append(dict_time["{}".format(key)])
        data_4_pd["triggers"].append(num_trig_per_sub[key])
        data_4_pd["error_charge"].append(  (((hit_charge_per_sub[key]  )**(1/2)/ num_trig_per_sub[key])**2+((num_trig_per_sub[key]**(1/2))/num_trig_per_sub[key]**2*hit_charge_per_sub[key])**2)**(1/2) )
        data_4_pd["error_noise"].append(( ((hit_noise_per_sub[key]  )**(1/2)/ num_trig_per_sub[key]   )**2+((num_trig_per_sub[key]**(1/2))/num_trig_per_sub[key]**2*hit_noise_per_sub[key])**2)**1/2  )


    else:
        print ("Can't save for suburn {}".format(key))

TypeError: tuple indices must be integers or slices, not str

In [19]:
df_sub=pd.DataFrame(data_4_pd)
df_sub['time'] = pd.to_datetime(df_sub['time'],unit='s')


NameError: name 'data_4_pd' is not defined

In [ ]:
df_sub=df_sub.set_index("time")

In [ ]:
df_sub["hour"]=df_sub.index.hour
df_sub["day"]=df_sub.index.date

In [ ]:
df_sub.to_pickle("vs_time_analysys_save/351")

In [ ]:
fig = px.scatter(df_sub,y="charge_hits",error_y="error_charge",hover_data=["triggers"])

In [ ]:
fig.show()

In [ ]:
fig = px.box(df_sub, x="hour", y="charge_hits",hover_data=["triggers"])

In [ ]:
fig.show()

In [ ]:
fig = px.box(df_sub, x="day", y="charge_hits",hover_name="sub_num",hover_data=["triggers"])
fig.show()

In [ ]:
fig = px.scatter(df_sub,y="noise_hits",error_y="error_noise")
fig.show()

In [ ]:
fig = px.box(df_sub, x="hour", y="noise_hits")
fig.show()

In [ ]:
fig = px.box(df_sub, x="day", y="noise_hits")
fig.show()

In [ ]:
fig = px.scatter(df_sub, y="triggers")
fig.show()

In [ ]:
df_sub=pd.read_pickle("/home/alb/Desktop/elaborazioni_e_dati/analisi_run/vs_time_analysys_save/351")
df_sub

In [ ]:
df_sub["sig/noise"]=df_sub.charge_hits/df_sub.noise_hits

In [ ]:
df_sub

In [ ]:
fig = px.scatter(df_sub,x="hour",y="sig/noise")
fig.show()